In [2]:
import nltk
from nltk.stem.lancaster import LancasterStemmer
stemmer = LancasterStemmer()

import numpy as np
import pandas as pd
import tflearn
import tensorflow as tf
import random

Instructions for updating:
non-resource variables are not supported in the long term
curses is not supported on this machine (please install/reinstall curses for an optimal experience)


In [3]:
# import contextual chat-bot intents
import json
with open('intents.json') as json_data:
    intents = json.load(json_data)

In [4]:
print(intents)

{'intents': [{'tag': 'greeting', 'patterns': ['Hi there', 'Is anyone there?', 'Hello', 'Hi'], 'responses': ['Hello there!', 'Hi, how can I help you?', 'Hi!', 'Hello!']}, {'tag': 'options', 'patterns': ['How can you help me?', 'What you can do?', 'I need help', 'What to enter?', 'I want your help', 'I am not feeling well'], 'responses': ['I can help you get an initial diagnosis of for health issues based on your symptoms. Please enter your symptoms to continue']}, {'tag': 'greetings_bye', 'patterns': ['Thanks for your help', 'Bye', 'Goodbye', 'Thanks', 'Thank you', 'okay', 'ok'], 'responses': ['No problem! It was a pleasure helping you', 'Bye-bye! Hope I able to help you :)', 'Goodbye! Hope you to see you again!', 'No problem :) I hope I was able to solve your problem!']}, {'tag': 'information', 'patterns': ['Who are you', 'Are you a doctor', 'Do you have a degree', 'Can you tell me about you', 'How do you know'], 'responses': ['I am Aimee! Your personal healthcare companion!', "I am no

In [10]:
# Verification that everything has been imported correctly
print(type(intents))
print(type(intents['intents']))
print(intents['intents'][0])
print(type(intents['intents'][0]))
print(intents['intents'][0]['tag'])

<class 'dict'>
<class 'list'>
{'tag': 'greeting', 'patterns': ['Hi there', 'Is anyone there?', 'Hello', 'Hi'], 'responses': ['Hello there!', 'Hi, how can I help you?', 'Hi!', 'Hello!']}
<class 'dict'>
greeting


In [4]:
words = []
classes = []
documents = []
ignore_words = ['?']

for intent in intents['intents']:
    for pattern in intent['patterns']:
        w = nltk.word_tokenize(pattern)
        words.extend(w)
        documents.append((w,intent['tag']))
        if intent['tag'] not in classes:
            classes.append(intent['tag'])
            
# stem words and remove duplicates
words = [stemmer.stem(w.lower()) for w in words if w not in ignore_words]
words = sorted(list(set(words)))

#remove duplicate classes
classes = sorted(list(set(classes)))

print(len(documents)," documents")
print(len(classes), " classes",classes)
print(len(words)," unique stemmed words",words)

29  documents
5  classes ['general', 'greeting', 'greetings_bye', 'information', 'options']
48  unique stemmed words [',', 'a', 'about', 'am', 'anyon', 'ar', 'bet', 'bye', 'can', 'degr', 'do', 'doct', 'ent', 'feel', 'for', 'get', 'good', 'goodby', 'hav', 'hello', 'help', 'hi', 'how', 'i', 'il', 'in', 'is', 'know', 'low', 'me', "n't", 'nee', 'not', 'ok', 'okay', 'pain', 'pleas', 'sick', 'tel', 'thank', 'ther', 'to', 'want', 'wel', 'what', 'who', 'yo', 'you']


In [14]:
training = []
output = []

output_empty = [0]*len(classes)

for doc in documents:
    bag = []
    pattern_words = doc[0]
    pattern_words = [stemmer.stem(word.lower()) for word in pattern_words]
    for w in words:
        bag.append(1) if w in pattern_words else bag.append(0)
        
    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1
    
    training.append([bag,output_row])
    
random.shuffle(training)
training = np.array(training)

train_x = list(training[:,0])
train_y = list(training[:,1])

C:\Users\Shreyan Das\AppData\Local\Temp\ipykernel_10436\743259283.py:19: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  training = np.array(training)


In [19]:
tf.compat.v1.reset_default_graph()
net = tflearn.input_data(shape = [None, len(train_x[0])])
net = tflearn.fully_connected(net,8)
net = tflearn.fully_connected(net,8)
net = tflearn.fully_connected(net,len(train_y[0]),activation='softmax')
net = tflearn.regression(net)

model = tflearn.DNN(net,tensorboard_dir='tflearn_logs')
model.fit(train_x, train_y,n_epoch=1000,batch_size=8,show_metric=True)
model.save('model.tflearn')

Training Step: 3999  | total loss: 0.00882 | time: 0.007s
| Adam | epoch: 1000 | loss: 0.00882 - acc: 0.9999 -- iter: 24/29
Training Step: 4000  | total loss: 0.00895 | time: 0.010s
| Adam | epoch: 1000 | loss: 0.00895 - acc: 0.9999 -- iter: 29/29
--
INFO:tensorflow:D:\My Documents\COLLEGE\Major Project\Code\ML Model\model.tflearn is not in all_model_checkpoint_paths. Manually adding it.


In [20]:
import pickle
pickle.dump({'words':words,'classes':classes,'train_x':train_x,'train_y':train_y},open("training_data","wb"))

### Test

In [7]:
import pickle

In [8]:
data = pickle.load(open("training_data","rb"))

words = data["words"]
classes = data["classes"]
train_x = data["train_x"]
train_y = data["train_y"]


with open("intents.json") as json_data:
    intents = json.load(json_data)

net = tflearn.input_data(shape = [None, len(train_x[0])])
net = tflearn.fully_connected(net,8)
net = tflearn.fully_connected(net,8)
net = tflearn.fully_connected(net,len(train_y[0]),activation='softmax')
net = tflearn.regression(net)

model = tflearn.DNN(net,tensorboard_dir='tflearn_logs')
model.load('model.tflearn')

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
INFO:tensorflow:Restoring parameters from D:\My Documents\COLLEGE\Major Project\Code\ML Model\model.tflearn
